# Pedestrian example of subscribing to a DataSet

It is possible to *subscribe* to a dataset. Subscribing means adding a function to the dataset and having the dataset call that function every time a result is added to the dataset (or more rarely, see below).

### Call signature

The subscribing function must have the following call signature:
```
fun(results: List[Tuple[Value]], length: int,
    state: Union[MutableSequence, MutableMapping]) -> None:
    """
    Args:
        results: A list of tuples where each tuple holds the results inserted into the dataset.
            For two scalar parameters, X and Y, results might look like [(x1, y1), (x2, y2), ...]
        length: The current length of the dataset.
        state: Any mutable sequence/mapping that can be used to hold information from call to call.
            In practice a list or a dict.
    """
```
Below we provide an example function that counts the number of times a voltage has exceeded a certain limit.

### Frequency

Since calling the function **every** time an insertion is made may be too frequent, a `min_wait` and a `min_count` argument may be provided when subscribing. The dataset will then only call the function upon inserting a result
if `min_wait` seconds have elapsed since the last call (or the start of the subscription, in the time before the first call) AND `min_count` results have been added to the dataset since the last call (or the start of the subscription). All the results added in the meantime are queued and passed to the function in one go.

### Order

The subscription must be set up **after** all parameters have been added to the dataset.

In [1]:
import qcodes

In [2]:
qcodes.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\jenielse\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [3]:
from qcodes import initialise_database, load_or_create_experiment, \
    new_data_set, ParamSpec
import numpy as np
from time import sleep

## Example 1: A notification

We imagine scanning a frequency and reading out a noisy voltage. When the voltage has exceeded a threshold 5 times, we want to receive a warning.

In [4]:
initialise_database()
exp = load_or_create_experiment(experiment_name="subscription_tutorial", sample_name="no_sample")

In [5]:
dataSet = new_data_set("test",
                       exp_id=exp.exp_id,
                       specs=[ParamSpec("x", "numeric", unit='Hz'),
                              ParamSpec("y", "numeric", unit='V')])
dataSet.mark_started()

[]
()
[ParamSpecBase('x', 'numeric', '', 'Hz'), ParamSpecBase('y', 'numeric', '', 'V')]
(ParamSpecBase('x', 'numeric', '', 'Hz'), ParamSpecBase('y', 'numeric', '', 'V'))
[ParamSpecBase('x', 'numeric', '', 'Hz'), ParamSpecBase('y', 'numeric', '', 'V')]
(ParamSpecBase('x', 'numeric', '', 'Hz'), ParamSpecBase('y', 'numeric', '', 'V'))


In [6]:
def threshold_notifier(results, length, state):
    if len(state) > 4:
        print(f'At step {length}: The voltage exceeded the limit 5 times! ')
        state.clear()
    for result in results:
        if result[1] > 0.8:
            state.append(result[1])

In [7]:
# now perform the subscription
# since this is important safety info, we want our callback function called
# on EVERY insertion
sub_id = dataSet.subscribe(threshold_notifier, min_wait=0, min_count=1, state=[])

NEW.x,NEW.y


In [8]:
for x in np.linspace(100, 200, 150):
    y = np.random.randn()
    dataSet.add_result({"x": x, "y": y})

At step 24: The voltage exceeded the limit 5 times! 
At step 65: The voltage exceeded the limit 5 times! 
At step 102: The voltage exceeded the limit 5 times! 


In [9]:
dataSet.unsubscribe_all()

## Example 2: ASCII Plotter

Not the most useful example for real life, but indeed what every kid on the block seems to be demanding.

In [10]:
dataSet = new_data_set("test", exp_id=exp.exp_id,
                       specs=[ParamSpec("blip", "numeric", unit='bit'),
                              ParamSpec("blop", "numeric", unit='bit')])
dataSet.mark_started()

[]
()
[ParamSpecBase('blip', 'numeric', '', 'bit'), ParamSpecBase('blop', 'numeric', '', 'bit')]
(ParamSpecBase('blip', 'numeric', '', 'bit'), ParamSpecBase('blop', 'numeric', '', 'bit'))
[ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')]
(ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit'))


In [11]:
def print_subscriber(results, lenght, state):
    print(f"result {results}")
    print(f"lenght {lenght}")
    print(f"state {state}")

In [12]:
def ASCII_plotter_5bit(results, length, state):
    """
    Glorious 5-bit signal plotter
    
    Digitises the range (-1, 1) with 4 bits and plots it
    in stdout. Crashes and burns if given data outside that
    interval.
    """
    for result in results:
        plotline = ['.'] * 32
        yvalue = result[0]
        yvalue += 1
        yvalue /= 2
        yvalue = int(yvalue*31)
        plotline[yvalue] = 'O'
        print(''.join(plotline))
        

In [13]:
sub_id = dataSet.subscribe(ASCII_plotter_5bit, min_wait=0, min_count=3, state=[])

NEW.blop,NEW.blip


In [24]:
from qcodes.dataset.descriptions.versioning.v0 import InterDependencies
from qcodes.dataset.descriptions.versioning.converters import old_to_new, new_to_old
from qcodes.dataset.descriptions.dependencies import InterDependencies_

In [15]:
specs=[ParamSpec("blip", "numeric", unit='bit'),
                              ParamSpec("blop", "numeric", unit='bit')]

In oldstyle interdependencies the parameters are ordered by insertion as they are just storred in a tuple of paramspecs

In [52]:
a = InterDependencies(*specs)
a

InterDependencies(ParamSpec('blip', 'numeric', '', 'bit', inferred_from=[], depends_on=[]), ParamSpec('blop', 'numeric', '', 'bit', inferred_from=[], depends_on=[]))

In [65]:
a.paramspecs

(ParamSpec('blip', 'numeric', '', 'bit', inferred_from=[], depends_on=[]),
 ParamSpec('blop', 'numeric', '', 'bit', inferred_from=[], depends_on=[]))

In [56]:
for s in a.paramspecs:
    print(s)

ParamSpec('blip', 'numeric', '', 'bit', inferred_from=[], depends_on=[])
ParamSpec('blop', 'numeric', '', 'bit', inferred_from=[], depends_on=[])


In [66]:
id_  = old_to_new(a)

[ParamSpecBase('blip', 'numeric', '', 'bit'), ParamSpecBase('blop', 'numeric', '', 'bit')]
(ParamSpecBase('blip', 'numeric', '', 'bit'), ParamSpecBase('blop', 'numeric', '', 'bit'))


In [67]:
id_

InterDependencies_(dependencies={}, inferences={}, standalones=frozenset({ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')}))

In [68]:
standls = id_.standalones

In [69]:
standls

frozenset({ParamSpecBase('blip', 'numeric', '', 'bit'),
           ParamSpecBase('blop', 'numeric', '', 'bit')})

However, in the new style they are storred as a set. The set has no guaranteed iteration order

In [70]:
standls.__repr__()

"frozenset({ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')})"

In [71]:
standls.__str__()

"frozenset({ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')})"

In [72]:
for s in standls:
    print(s)

ParamSpecBase('blop', 'numeric', '', 'bit')
ParamSpecBase('blip', 'numeric', '', 'bit')


This means that get_parameres may return the parameters in any order.

Internally the sqlite statment will setup a trigger that returns the data in the order given by get_parameters

In [49]:
dataSet.get_parameters()

[ParamSpec('blop', 'numeric', '', 'bit', inferred_from=[], depends_on=[]),
 ParamSpec('blip', 'numeric', '', 'bit', inferred_from=[], depends_on=[])]

In [16]:
dataSet.description

RunDescriber(InterDependencies_(dependencies={}, inferences={}, standalones=frozenset({ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')})))

In [17]:
dataSet._interdeps

InterDependencies_(dependencies={}, inferences={}, standalones=frozenset({ParamSpecBase('blop', 'numeric', '', 'bit'), ParamSpecBase('blip', 'numeric', '', 'bit')}))